# ED2 Jobs


## Background

The Ecosystem Demography Biosphere Model (ED2) is an integrated terrestrial biosphere model incorporating hydrology, land-surface biophysics, vegetation dynamics, and soil carbon and nitrogen biogeochemistry (Longo et al. 2019;Medvigy et al., 2009). Like its predecessor, ED (Moorcroft et al., 2001), ED2 uses a set of size- and age-structured partial differential equations that track the changing structure and composition of the plant canopy. With the ED2 model, in contrast to conventional biosphere models in which ecosystems with climatological grid cells are represented in a highly aggregated manner, the state of the aboveground ecosystem is described by the density of trees of different sizes and how this varies across horizontal space for a series of plant functional types. For more details, please go [here](https://github.com/EDmodel/*ED2*).

## Run ED2 jobs on HPC cluster

This notebook run ED2 jobs on HPC clusters. You can submit jobs for tonzi, harvest, umbs and santarem. We also have the ed binary sitting inside this docker container. So, feel free to run the job on localhost too.

## Prerequisites
The following modules/imports are necessary to run this notebook. The pip modules 'paramiko' and 'ipywidgets' are already installed in this docker container.

In [23]:
from IPython.display import display, Javascript
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import ipywidgets as widgets
import json
from IPython.display import display
import getpass
import paramiko
import stat
import os
import resource
import time as timer
import subprocess
from urllib.parse import urlencode
import IPython
import random
#from ED2_helpers import *

%run "Ed2_helpers.ipynb"

## Generating the Cluster Configuration File
This segment generates the "cluster.json" file, which holds information about each HPC cluster. Initially, a sample file with details of known clusters is created for first-time users. You're welcome to adjust the list as needed.

In [24]:
# Define the file path
file_path = "clusters.json"

# Check if the file already exists
if not os.path.exists(file_path):
    # File does not exist, so create it
    with open(file_path, "w") as f:
        # Write some initial content if neededs
        f.write('''
{
  "clusters": [
    {"hostname": "localhost", "batch_jobs":[], "queues":[], "modules_to_load": "", "pre_run_command": "", "apptainer_binary_command": "", "post_run_command": ""},
    {"hostname": "cc-login.campuscluster.illinois.edu", "batch_jobs": ["slurm"], "queues" : ["secondary", "IllinoisComputes", "IllinoisComputes-GPU"], "modules_to_load": "singularity", "pre_run_command": "", "apptainer_binary_command": "singularity exec", "post_run_command": ""},
    {"hostname": "login2.stampede2.tacc.utexas.edu", "batch_jobs": ["slurm", "Kubernetes scheduler", "LSF"], "queues" : ["development", "normal","skx-normal", "skx-large", "icx-normal", "skx-dev"], "modules_to_load": "tacc-apptainer", "pre_run_command": "", "apptainer_binary_command": "singularity exec", "post_run_command": ""},
    {"hostname": "login.delta.ncsa.illinois.edu", "batch_jobs": ["slurm"], "queues" : ["cpu", "cpu-interactive","cpu-preempt"], "modules_to_load": "", "pre_run_command": "ulimit -s unlimited", "apptainer_binary_command": "apptainer run", "post_run_command": ""}
  ]
}
''')

## UI component for HPC details

In [25]:
with open('clusters.json', 'r') as json_file:
    clusters_data = json.load(json_file)
    cluster_options = [(cluster["hostname"], cluster["hostname"]) for cluster in clusters_data["clusters"]]

# Create a dictionary to store batch job options for each cluster
batch_jobs_dict = {cluster["hostname"]: cluster["batch_jobs"] for cluster in clusters_data["clusters"]}

# Create a dictionary to store batch job options for each cluster
queues_dict = {cluster["hostname"]: cluster["queues"] for cluster in clusters_data["clusters"]}
cluster_input = widgets.Dropdown(options=cluster_options, description="Cluster:")

username_input = widgets.Text(description="User name:")
user_password = widgets.Password(placeholder='Enter password',
    description='Password:',
    disabled=False
)
user_pkey = widgets.Password(placeholder='Enter path to private key',
    description='Private key path:',
    disabled=False
)
user_acc = widgets.Text(value="campusclusterusers", description="Account:")
batch_job_input = widgets.Dropdown(description="Batch job:")
queue_input = widgets.Combobox(
    placeholder='',
    options=[],
    ensure_option=True,  # True: restrict input to options; False: allow any input
    disabled=False
)
queue = widgets.HBox([widgets.Label(value="Queue name:"), queue_input])

# Set initial options for batch_job_input based on the first cluster
initial_cluster = cluster_options[0][0]
batch_job_input.options = batch_jobs_dict.get(initial_cluster, [])
queue_input.options = queues_dict.get(initial_cluster, [])

ed_binary_singularity_input = widgets.HBox([widgets.Label(value="Path to binary/singularity:"), widgets.Text(description="", value="${HOME}/ed2.sif") ])
job_name_input = widgets.Text(value="ED2IN", description="Job name:")
num_nodes_input = widgets.IntText(value=1, description="No of nodes:")
runtime_input = widgets.Text(value="00:15:00", description="Runtime:")

hpc_accordion = widgets.Accordion(children=[widgets.VBox([cluster_input, username_input, user_password, user_pkey, user_acc, batch_job_input, queue, ed_binary_singularity_input, job_name_input, num_nodes_input, runtime_input])])
hpc_accordion.set_title(0, 'HPC Parameters')
display(hpc_accordion)

Accordion(children=(VBox(children=(Dropdown(description='Cluster:', options=(('cc-login.campuscluster.illinois…

## UI component for ED path parameters

In [26]:
work_folder_input = widgets.Text(placeholder="$HOME/ed/UUID", value="${HOME}/ed-demo", description="Work folder:")
ED2IN_path_input = widgets.Text(description="ED2IN path:", value="ED2IN-umbs.bg")
header_file_input = widgets.Text(description="Header file path:", value="$HOME/ED-2.2_StartKit/ED2_InputData/SiteData/Santarem_Km83/MeteoDriver/Santarem_Km83_HEADER")
ed_path_accordion = widgets.Accordion(children=[widgets.VBox([ED2IN_path_input, work_folder_input, header_file_input])])
ed_path_accordion.set_title(0, 'ED Path Parameters')
display(ed_path_accordion)

Accordion(children=(VBox(children=(Text(value='ED2IN-umbs.bg', description='ED2IN path:'), Text(value='${HOME}…

## UI component for ED variables parameters

In [27]:
met_driver_input = widgets.Text(value="", description="MET driver:")
var_options = ['NL%FFILOUT', 'NL%SFILOUT', 'NL%GFILOUT', 'NL%SFILIN', 'NL%VEG_DATABASE', 'NL%SOIL_DATABASE', 'NL%LU_DATABASE', 'NL%THSUMS_DATABASE', 'NL%ED_MET_DRIVER_DB', 'NL%SOILSTATE_DB', 'NL%SOILDEPTH_DB']
# Create the "+" button
add_button = widgets.Button(description="Add more variable")
add_button.on_click(add_dropdown)

# Create the "-" button to remove dropdowns
remove_button = widgets.Button(description="Remove variable")
remove_button.on_click(remove_dropdown)

# Create the initial dropdown widget
initial_dropdown = create_dropdown()

# Create a container to hold all dropdowns
var_dropdowns = widgets.VBox([initial_dropdown])


ed_vars_accordion = widgets.Accordion(children=[widgets.VBox([met_driver_input, var_dropdowns, widgets.HBox([add_button, remove_button])])])
ed_vars_accordion.set_title(0, 'ED Vars')
display(ed_vars_accordion)

Accordion(children=(VBox(children=(Text(value='', description='MET driver:'), VBox(children=(HBox(children=(Dr…

In [28]:
## UI component for Submit button

In [29]:
submit_button = widgets.Button(description="Submit Job")
submit_button.on_click(on_submit_button_click)

submit_job_accordion = widgets.Accordion(children=[widgets.VBox([submit_button])])
submit_job_accordion.set_title(0, 'Submit Job')
display(submit_job_accordion)
#display(input)
display(output)

Accordion(children=(VBox(children=(Button(description='Submit Job', style=ButtonStyle()),)),), titles=('Submit…

Output()

In [30]:
## UI component for Next button

In [31]:
next_button = widgets.Button(description="Next")
next_button.on_click(redirect_to_notebook)

next_job_accordion = widgets.Accordion(children=[widgets.VBox([next_button])])
next_job_accordion.set_title(0, 'See Job status')
display(next_job_accordion)

Accordion(children=(VBox(children=(Button(description='Next', style=ButtonStyle()),)),), titles=('See Job stat…